In [1]:
import pandas as pd
import numpy as np
import re
import prepare
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt 


Use this to drop normalized columns if necessary:


    columns = []
    for i in df.columns:
        columns.append(i)
    columns = str(columns)
    normalized_columns = re.findall(r'(smart_\d+_normalized)', columns)
    df.drop(columns=normalized_columns, inplace=True)"""

In [2]:
df = pd.read_csv('hard_drives_smart_5.csv')

In [3]:
df.shape

(169073, 11)

In [4]:
df.drop(columns=('Unnamed: 0'), inplace = True)

In [5]:
df.head(10)

,serial_number,model,capacity_bytes,max(failure),max(smart_9_raw),max(smart_5_raw),max(smart_187_raw),max(smart_188_raw),max(smart_197_raw),max(smart_198_raw)
0,PL1311LAG1SJAA,Hitachi HDS5C4040ALE630,4000787030016,0,43819.0,0.0,NaN,NaN,0.0,0.0
1,Z305KB36,ST4000DM000,4000787030016,0,31045.0,0.0,0.0,0.0,0.0,0.0
2,MJ0351YNG9MZXA,Hitachi HDS5C3030ALA630,3000592982016,0,41668.0,0.0,NaN,NaN,0.0,0.0
3,ZA11NHSN,ST8000DM002,8001563222016,0,26284.0,0.0,0.0,0.0,0.0,0.0
4,MJ1311YNG2ZSEA,Hitachi HDS5C3030ALA630,3000592982016,0,47994.0,0.0,NaN,NaN,0.0,0.0
5,Z305D5VF,ST4000DM000,4000787030016,0,31883.0,0.0,0.0,0.0,0.0,0.0
6,Z3015L2E,ST4000DM000,4000787030016,0,36768.0,0.0,0.0,0.0,0.0,0.0
7,PL1331LAHDYJYH,HGST HMS5C4040BLE640,4000787030016,0,30144.0,0.0,NaN,NaN,0.0,0.0
8,Z304LAHP,ST4000DM000,4000787030016,0,33885.0,0.0,0.0,0.0,0.0,0.0
9,Z302B0GK,ST4000DM000,4000787030016,0,36443.0,0.0,0.0,0.0,0.0,0.0


In [6]:
prepare.prepare(df)

,serial_number,manufacturer,model,capacity_gigabytes,failure,drive_age_in_years,reallocated_sectors_count,reported_uncorrectable_errors,command_timeout,current_pending_sector_count,uncorrectable_sector_count
0,PL1311LAG1SJAA,Hitachi,Hitachi HDS5C4040ALE630,4000.8,0,5.0,0.0,NaN,NaN,0.0,0.0
1,Z305KB36,Seagate,ST4000DM000,4000.8,0,3.5,0.0,0.0,0.0,0.0,0.0
2,MJ0351YNG9MZXA,Hitachi,Hitachi HDS5C3030ALA630,3000.6,0,4.8,0.0,NaN,NaN,0.0,0.0
3,ZA11NHSN,Seagate,ST8000DM002,8001.6,0,3.0,0.0,0.0,0.0,0.0,0.0
4,MJ1311YNG2ZSEA,Hitachi,Hitachi HDS5C3030ALA630,3000.6,0,5.5,0.0,NaN,NaN,0.0,0.0
5,Z305D5VF,Seagate,ST4000DM000,4000.8,0,3.6,0.0,0.0,0.0,0.0,0.0
6,Z3015L2E,Seagate,ST4000DM000,4000.8,0,4.2,0.0,0.0,0.0,0.0,0.0
7,PL1331LAHDYJYH,Hitachi,HGST HMS5C4040BLE640,4000.8,0,3.4,0.0,NaN,NaN,0.0,0.0
8,Z304LAHP,Seagate,ST4000DM000,4000.8,0,3.9,0.0,0.0,0.0,0.0,0.0
9,Z302B0GK,Seagate,ST4000DM000,4000.8,0,4.2,0.0,0.0,0.0,0.0,0.0


In [7]:
df['drive_age_in_years'].describe()

count    161975.000000
mean          2.720529
std           1.525822
min           0.000000
25%           1.600000
50%           2.600000
75%           4.000000
max          10.300000
Name: drive_age_in_years, dtype: float64

## Manufacturers

- There are 7 different manufacturers
- Seagate and Hitachi make up 97% of all hard drives
- There are 103 unqiue  hard drive models. 43 of these models have less than 10 hard drives. 72 models have less than 100.

In [8]:
df['manufacturer'].nunique()

7

In [9]:
# How many hard drives per manufacturer?
df['manufacturer'].value_counts()

Seagate            110097
Hitachi             53473
Western Digital      2826
Toshiba              2604
Dell                   60
Samsung                11
Unknown                 2
Name: manufacturer, dtype: int64

In [10]:
(110097+53473)/169073 *100

96.74519290483991

In [11]:
# How many unique models?
df['model'].nunique()

103

In [12]:
# How many hard drives models with less than 10 hard drives?
(df['model'].value_counts()  < 10).sum()

43

In [13]:
# How many hard drives by individual model?
df['model'].value_counts()

ST12000NM0007                          39779
ST4000DM000                            36802
ST8000NM0055                           17305
HGST HMS5C4040BLE640                   17011
HGST HUH721212ALN604                   12036
ST8000DM002                            10345
HGST HMS5C4040ALE640                    8813
Hitachi HDS5C3030ALA630                 4563
Hitachi HDS722020ALA330                 4503
Hitachi HDS5C4040ALE630                 2639
ST6000DX000                             1942
HGST HUH721212ALE600                    1564
ST10000NM0086                           1329
TOSHIBA MG07ACA14TA                     1328
WDC WD30EFRX                            1167
HGST HUH728080ALE600                    1102
Hitachi HDS723030ALA640                 1000
ST500LM012 HN                            804
TOSHIBA MQ01ABF050                       589
WDC WD60EFRX                             471
ST4000DM001                              425
TOSHIBA MQ01ABF050M                      410
WDC WD5000

In [14]:
# Average drive age by manufacturer
manu = df[['manufacturer', 'drive_age_in_years']]
manu.groupby('manufacturer',sort=True).mean()

,drive_age_in_years
manufacturer,
Dell,NaN
Hitachi,3.057180
Samsung,0.300000
Seagate,2.555197
Toshiba,1.611698
Unknown,1.700000
Western Digital,3.765322


### Failures vs. total by model number

In [15]:
agg = df.groupby(['model']).agg({'failure' : 'sum', 'model' : 'count', 'drive_age_in_years':'mean'})

In [16]:
agg.rename(columns={'failure':'failures', 'model':'total_count'}, inplace=True)

In [17]:
agg['failure_rate_percent'] = agg['failures'] / agg['total_count'] * 100

In [21]:
agg.sort_values(by=['failure_rate_percent'], ascending = False)

,failures,total_count,drive_age_in_years,failure_rate_percent
model,,,,
WDC WD800LB,1,1,10.300000,100.000000
WDC WD3200BEKT,1,1,2.500000,100.000000
WDC WD800JD,1,1,4.800000,100.000000
WDC WD800JB,4,4,6.475000,100.000000
WDC WD800BB,5,5,9.240000,100.000000
WDC WD800AAJS,14,14,4.078571,100.000000
Samsung SSD 850 EVO 1TB,10,10,0.090000,100.000000
ST3500320AS,1,1,4.900000,100.000000
ST250LT007,4,4,2.600000,100.000000


In [19]:
agg.sort_values(by=['failure_rate_percent'], ascending = False)

,failures,total_count,drive_age_in_years,failure_rate_percent
model,,,,
WDC WD800LB,1,1,10.300000,100.000000
WDC WD3200BEKT,1,1,2.500000,100.000000
WDC WD800JD,1,1,4.800000,100.000000
WDC WD800JB,4,4,6.475000,100.000000
WDC WD800BB,5,5,9.240000,100.000000
WDC WD800AAJS,14,14,4.078571,100.000000
Samsung SSD 850 EVO 1TB,10,10,0.090000,100.000000
ST3500320AS,1,1,4.900000,100.000000
ST250LT007,4,4,2.600000,100.000000
